# Spotify Database Cleaning

We will clean the Spotify database that we created in the create_spotify_db file. Because of how we built the genres, tracks_artists, and genres_artists tables, these will not require any cleaning. However, we will need to clean the artists and tracks tables. First, let's import the necessary packages, open a connection to the database and create a cursor, and set display options for pandas.

In [1]:
import sqlite3
import pandas as pd
import statsmodels.formula.api as smf
from funcs import execute_print

conn = sqlite3.connect('spotify.db')
c = conn.cursor()

#Set max rows and limit floats to 2 decimal places in pandas
pd.options.display.max_rows = 10
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## I. Artists Table

First, let's look at the artists table itself.

In [2]:
select_artists = """
    SELECT *
    FROM artists
    LIMIT 50;
"""
artists = pd.read_sql_query(select_artists, conn)
artists

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2,[],Ioannis Panoutsopoulos,0
...,...,...,...,...,...
45,0VLMVnVbJyJ4oyZs2L3Yl2,71,['carnaval cadiz'],Las Viudas De Los Bisabuelos,6
46,0dt23bs4w8zx154C5xdVyl,63,['carnaval cadiz'],Los De Capuchinos,5
47,0pGhoB99qpEJEsBQxgaskQ,64,['carnaval cadiz'],Los “Pofesionales”,7
48,3HDrX2OtSuXLW5dLR85uN3,53,['carnaval cadiz'],Los Que No Paran De Rajar,6


And let's look at some information about the columns.

In [3]:
artists_info = "PRAGMA table_info(artists);"
execute_print(artists_info, c)

(0, 'id', 'TEXT', 1, None, 1)
(1, 'followers', 'INTEGER', 0, None, 0)
(2, 'genres', 'TEXT', 0, None, 0)
(3, 'name', 'TEXT', 0, None, 0)
(4, 'popularity', 'INTEGER', 0, None, 0)


We have two integer columns, followers and popularity. We should check to make sure that these all contain values that are in the correct range. We should also check generally for missing data and duplicate rows in the whole table.

### 1. Check for out-of-range values

The popularity value for an artist should not be less than 0 or greater than 100 and the follower count should not be less than 0 [1]. We can check this by querying for any distinct popularity and followers values that are outside of these ranges.

In [4]:
check_popularity_followers = """
    SELECT DISTINCT popularity
    FROM artists
    WHERE popularity < 0 OR popularity > 100
    OR followers < 0;
"""
execute_print(check_popularity_followers, c)

No results


This pulled up no results, so we do not have any out-of-range values.

### 2. Check for Missing Data

Even if we don't have any out-of-range data for the integer columns, we might still have missing data in any column, such as blank or NULL values. Let's query the database for any blank or NULL values in any of the columns.

In [5]:
check_artists_missing = """
    SELECT *
    FROM artists
    WHERE followers IS NULL OR followers = ''
    OR genres IS NULL OR genres = ''
    OR name IS NULL OR name = ''
    OR popularity IS NULL OR popularity = '';
"""
artists_missing = pd.read_sql_query(check_artists_missing, conn)

#Increase rows displayed
pd.options.display.max_rows = 20

artists_missing

,id,followers,genres,name,popularity
0,7F71W80jaXFARK7hBjsDI2,,['czech pop'],Marcell,36
1,3MLHJz04KmEVzCTPclzkEm,,['czech pop'],Niko,21
2,0cqZsULDZdJTGA4Zqh8Ckv,,[],Savzilla,0
3,0BuknWzKujyc9HfZ1V50Uk,,[],Duck Doja,0
4,6ltU5gIDLmWNYaVNHnll5G,,['mexican electronic'],Zofa,0
5,7C9nWRMbRqpPUuKh2OEw9n,,[],MHV,3
6,41c30F8zy5UCTSevbn0WfD,,['mexican electronic'],Broadband Star,0
7,1DK979aOesiZ4Vkus8txqu,,[],AmorArtis Orchestra & Johannes Somary,2
8,2lr0R5vHGfI0C489h0r6qV,,['dc indie'],Black Dog Prowl,16
9,0xkSOIeyeTILNIOZKyFgaP,,[],Band of the Fifteenth Field Artillery Regiment...,3


These artists are all missing follower counts. It is unclear exactly why these follower counts are missing. They could be missing completely at random (i.e., the missingness of the data has nothing to do with the value of the missing data or other observed data) [2]. This might happen if there were a random error in data entry or Spotify's process of generating this data. The data could also be missing at random (i.e., the missingness of the data is related to other observed data but not to the value of the missing data itself) [3]. For example, it looks like at least some of these artists belong to related genres (e.g., Czech pop, Mexican electronic). Data missing at random or missing completely at random can generally be dropped without biasing the rest of the data [2].

However, it is also possible that the data is missing not at random (i.e., the missingness is related to the value of the data itself) [3]. Dropping data that is missing not at random can bias the rest of the data [2]. For example, perhaps these artists all had no followers, and that somehow led to the data being missing. However, it seems unlikely in this case, as there are only 11 records out of over 1 million artists and there are likely many more Spotify artists with no followers.

In this particular situation, the missing follower counts and cause of the missingness are not too concerning because so few observations are missing. We could probably drop these rows without significantly biasing the data, but then we would lose a lot of other valuable information about these artists. We could fill all of them with 0, but that might not make sense for artists with higher popularity ratings. What might make more sense in this situaiton is imputing the median follower count for artists of the given artist's popularity level (e.g., Marcell has popularity of 36, so impute the median follower count for artists of popularity 36 as Marcell's follower count). Median would make sense for this, as it is more robust to outliers than the mean, and we do not know the distribution of follower counts. This is a relatively simple imputation method and won't be perfect, but it seems more realistic than imputing 0 for each artist.

SQLite doesn't have a built in median function, so this would be easier to do utilizing pandas to obtain the necessary medians. First, we need to read all artists into a pandas DataFrame. Then, from the artists_missing DataFrame we extract the popularity values and insert a new imputed_followers column with the median number of followers for artists at that artist's popularity level. Then, we use a loop to fill the followers column for each artist in SQLite with the imputed follower count. 

In [6]:
#Read all artists into a pandas DataFrame
select_all_artists = """
    SELECT *
    FROM artists;
"""
artists = pd.read_sql_query(select_all_artists, conn)

#Extract popularity values for artists missing followers
artist_pops = artists_missing['popularity'].values

#Create imputed_followers column and fill with median followers for
#artists at that artist's popularity level
artists_missing['imputed_followers'] = [
    artists[(artists['popularity'] == pop) & (artists['followers'] != '')]\
    ['followers'].median(axis=0) for pop in artist_pops]

#Fetch all artists with missing followers in SQL
c.execute(check_artists_missing)
artists_to_impute = c.fetchall()

#Fill followers column with imputed followers number using loop
fill_followers = """
    UPDATE artists
    SET followers = ?
    WHERE id = ?;
"""
check_followers = """
    SELECT *
    FROM artists
    WHERE id = ?;
"""

for artist, i in zip(artists_to_impute, range(11)):
    c.execute(fill_followers, (artists_missing['imputed_followers'][i], artist[0]))
    conn.commit()
    c.execute(check_followers, (artist[0],)) #Check results
    print(c.fetchall())

[('7F71W80jaXFARK7hBjsDI2', 4505, "['czech pop']", 'Marcell', 36)]
[('3MLHJz04KmEVzCTPclzkEm', 848, "['czech pop']", 'Niko', 21)]
[('0cqZsULDZdJTGA4Zqh8Ckv', 10, '[]', 'Savzilla', 0)]
[('0BuknWzKujyc9HfZ1V50Uk', 10, '[]', 'Duck Doja', 0)]
[('6ltU5gIDLmWNYaVNHnll5G', 10, "['mexican electronic']", 'Zofa', 0)]
[('7C9nWRMbRqpPUuKh2OEw9n', 89, '[]', 'MHV', 3)]
[('41c30F8zy5UCTSevbn0WfD', 10, "['mexican electronic']", 'Broadband Star', 0)]
[('1DK979aOesiZ4Vkus8txqu', 71, '[]', 'AmorArtis Orchestra & Johannes Somary', 2)]
[('2lr0R5vHGfI0C489h0r6qV', 524, "['dc indie']", 'Black Dog Prowl', 16)]
[('0xkSOIeyeTILNIOZKyFgaP', 89, '[]', 'Band of the Fifteenth Field Artillery Regiment, RCA', 3)]
[('4EqqnE0XMAcreVF84QGYJ0', 10, '[]', 'Robert Steven Williams', 0)]


These look like plausible follower counts for each artist's popularity level.

### 3. Check for duplicates

Finally, we should check for any duplicate artist IDs. We can do this by creating a CTE where we partition the rows by ID and assign a row number representing how many times each ID appears, then querying that CTE for any records with row number greater than 1. Any row numbers greater than 1 represent duplicate artist IDs.

In [7]:
duplicate_artists = """
    WITH duplicates AS (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY id ORDER BY id) row_num
        FROM artists)
    SELECT *
    FROM duplicates
    WHERE row_num > 1;
"""
execute_print(duplicate_artists, c)

No results


This returned no results, so we do not have any duplicate artist IDs.

## II. Tracks Table

First, let's look at the tracks table itself.

In [8]:
select_tracks = """
    SELECT *
    FROM tracks
    LIMIT 50;
"""
tracks = pd.read_sql_query(select_tracks, conn)
tracks

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],1922-02-22,0.65,0.45,0,-13.34,1,0.45,0.67,0.74,0.15,0.13,104.85,3,1922
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],1922-06-01,0.69,0.26,0,-22.14,1,0.96,0.80,0.00,0.15,0.66,102.01,1,1922
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],1922-03-21,0.43,0.18,1,-21.18,1,0.05,0.99,0.02,0.21,0.46,130.42,5,1922
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],1922-03-21,0.32,0.09,7,-27.96,1,0.05,0.99,0.92,0.10,0.40,169.98,3,1922
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],1922,0.40,0.16,3,-16.90,0,0.04,0.99,0.13,0.31,0.20,103.22,4,1922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,1kXWSsJkBVZ1jSoI8NnEDm,Marta,0,177693,0,['Dick Haymes'],1922,0.26,0.34,0,-9.37,1,0.03,0.99,0.02,0.33,0.31,101.10,4,1922
46,1l1Wk0nOkuMCzioN6l2yfJ,Carol of the Bells,0,286370,0,['Grandcubby Trio'],1922,0.38,0.75,10,-11.60,1,0.07,0.12,0.75,0.72,0.35,149.79,3,1922
47,1lia44teZBfbv0PnPkc5dK,Machinalement,0,145400,0,['Victor Boucher'],1922,0.46,0.19,4,-16.82,1,0.07,1.00,0.35,0.08,0.67,177.10,4,1922
48,1pGBOfY0PvpArBZT7GaUVK,Capítulo 2.19 - Banquero Anarquista,0,106000,0,['Fernando Pessoa'],1922-06-01,0.73,0.21,10,-22.14,1,0.96,0.77,0.00,0.56,0.73,110.86,5,1922


And let's look at some information about the columns.

In [9]:
tracks_info = "PRAGMA table_info(tracks);"
execute_print(tracks_info, c)

(0, 'id', 'TEXT', 1, None, 1)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'popularity', 'INTEGER', 0, None, 0)
(3, 'duration_ms', 'INTEGER', 0, None, 0)
(4, 'explicit', 'INTEGER', 0, None, 0)
(5, 'artists', 'TEXT', 0, None, 0)
(6, 'release_date', 'TEXT', 0, None, 0)
(7, 'danceability', 'REAL', 0, None, 0)
(8, 'energy', 'REAL', 0, None, 0)
(9, 'key', 'INTEGER', 0, None, 0)
(10, 'loudness', 'REAL', 0, None, 0)
(11, 'mode', 'INTEGER', 0, None, 0)
(12, 'speechiness', 'REAL', 0, None, 0)
(13, 'acousticness', 'REAL', 0, None, 0)
(14, 'instrumentalness', 'REAL', 0, None, 0)
(15, 'liveness', 'REAL', 0, None, 0)
(16, 'valence', 'REAL', 0, None, 0)
(17, 'tempo', 'REAL', 0, None, 0)
(18, 'time_signature', 'INTEGER', 0, None, 0)
(19, 'release_year', 'INTEGER', 0, None, 0)


As with the artists table, we have some integer columns, but also a lot of real number columns for many of the audio features for each track. We will break these into different groups to check whether all values are within range. As with the artists table, we should also check generally for missing data and duplicate rows in the whole table.

### 1. Check for out-of-range values

#### A. Popularity and duration

As with artists, tracks have a popularity ranging from 0 to 100 [4]. The duration_ms column also should not be less than or equal to 0. Let's query for any distinct values outside of these ranges.

In [10]:
check_popularity_duration = """
    SELECT DISTINCT popularity
    FROM tracks
    WHERE popularity < 0 OR popularity > 100
    OR duration_ms <= 0;
"""
execute_print(check_popularity_duration, c)

No results


This returned no results, so we have no out-of-range values.

#### B. Explicit, key, and mode

Next, the explicit, key, and mode columns each have a limited range of integer values. Explicit and mode are just binary, with 0 and 1 representing not explicit or explicit respectively for the explicit column [4] and minor or major respectively for the mode column [5]. Key is an integer from 0 to 11 representing one of the 12 standard keys and is -1 if no key is detected [5].

For each of these columns we can just select all the distinct values of each and visually check if any are out-of-range.

In [11]:
check_explicit = """
    SELECT DISTINCT explicit
    FROM tracks;
"""
execute_print(check_explicit, c)

(0,)
(1,)


In [12]:
check_key = """
    SELECT DISTINCT key
    FROM tracks;
"""
execute_print(check_key, c)

(0,)
(1,)
(7,)
(3,)
(5,)
(4,)
(6,)
(11,)
(2,)
(8,)
(10,)
(9,)


In [13]:
check_mode = """
    SELECT DISTINCT mode
    FROM tracks;
"""
execute_print(check_mode, c)

(1,)
(0,)


These queries all returned values that are within the correct ranges.

#### C. Danceability, energy, speechiness, acousticness, instrumentalness, liveness, valence, and loudness

Next, we have a series of audio features that are all real numbers ranging from 0 to 1--danceability, energy, speechiness, acousticness, instrumentalness, liveness, and valence [5]. We also have a loudness feature which should not be less than -60 [5]. Let's query the distinct values of all of these at once to see if they pull up any results that are out-of-range.

In [14]:
check_features = """
    SELECT DISTINCT danceability
    FROM tracks
    WHERE danceability < 0 OR danceability > 1
    OR energy < 0 OR energy > 1
    OR speechiness < 0 OR speechiness > 1
    OR acousticness < 0 OR acousticness > 1
    OR instrumentalness < 0 OR instrumentalness > 1
    OR liveness < 0 OR liveness > 1
    OR valence < 0 OR valence > 1
    OR loudness < -60;
"""
execute_print(check_features, c)

No results


There were no results for this query, so we do not have any out-of-range values for these columns.

#### D. Release

Next, we have release_date and release_year columns. The data claims to contain tracks from 1921 to 2020 [6], and because the release_year column was created from the release date column, we can just query the release_year column for any distinct years that are outside of this range.

In [15]:
check_release_year = """
    SELECT DISTINCT release_year
    FROM tracks
    WHERE release_year < 1921 OR release_year > 2020;
"""
execute_print(check_release_year, c)

(2021,)
(1900,)


This brought up 1900 and 2021, so let's look further at these records. First, let's look at records from 2021.

In [16]:
check_2021 = """
    SELECT *
    FROM tracks
    WHERE release_year = 2021;
"""
release_2021 = pd.read_sql_query(check_2021, conn)
release_2021

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,3nyP7Dgjiy1cay2HRD98I7,09 01 1966,3,242068,0,['Primiano Red'],2021-01-28,0.51,0.34,5,-11.77,0,0.03,0.78,0.00,0.19,0.32,150.14,3,2021
1,0mrBr6it6Zcbn3fzXfZWC3,Year 3000,26,163680,1,['JayTheKing'],2021-02-04,0.61,0.54,10,-9.35,0,0.10,0.10,0.00,0.12,0.18,146.92,4,2021
2,558D3GX2TjNixp21ZCjTvN,THE YEAR 3000,0,114888,0,"['The Fifth Reality', 'Benjamin']",2021-04-10,0.62,0.84,4,-14.33,0,0.10,0.36,0.41,0.67,0.78,140.06,4,2021
3,1I8I42gmNic7Jag290dVTM,Logical Brain - Year 3000 Mix,0,216705,0,['Electro Mann'],2021-04-09,0.74,0.73,9,-11.29,1,0.08,0.08,0.93,0.07,0.95,125.00,4,2021
4,5qzcKzMSBsHLrq3ldccNOd,Year 3000,0,120301,0,['Ellis Dews'],2021-02-24,0.71,0.94,2,-8.06,1,0.06,0.01,0.74,0.10,0.74,168.07,4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6276,5Pww1E3L6p0DnY9w8JnpE6,EXTENDO,57,166286,1,['Dizzy'],2021-01-22,0.81,0.59,2,-6.74,1,0.25,0.05,0.00,0.38,0.44,140.04,4,2021
6277,6g0RVZLxxL62W4YVMWW03a,Marek Hamšík,44,148447,0,"['Rasmus Gozzi', 'Viktor Klemming']",2021-04-08,0.75,0.93,11,-6.48,1,0.47,0.11,0.00,0.77,0.44,137.91,4,2021
6278,5n2dOPDtXgvZGJO0RqQj2P,Rich,57,170667,0,['Julia Alfrida'],2021-02-13,0.60,0.72,11,-5.97,0,0.22,0.02,0.00,0.09,0.36,169.94,4,2021
6279,0GWNh7kAj4Lq14UBxCPK5h,At Sunset,1,212851,0,['Oriental Music Zone'],2021-01-16,0.19,0.14,1,-23.37,1,0.04,0.95,0.93,0.16,0.04,113.44,3,2021


Without diving into all 6281 records, the data set appears to contains valid records from 2021. So, the stated date range of the data set (1921-2020) is likely just incorrect. Next, let's look at records from 1900.

In [17]:
check_1900 = """
    SELECT *
    FROM tracks
    WHERE release_year = 1900;
"""
release_1900 = pd.read_sql_query(check_1900, conn)
release_1900

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,74CSJTE5QQp1e4bHzm3wti,Maldita sea la primera vez,19,233920,0,['Los Pincheira del Sur'],1900-01-01,0.66,0.79,2,-4.89,1,0.03,0.14,0.00,0.16,0.96,142.00,4,1900


This brought up one record--'Maldita sea la primera vez' by Los Pincheira del Sur. A simple Google search reveals that this was actually released in [2014](https://www.youtube.com/watch?v=FVdTHRwFDVI), so let's correct it in both the release_date column and the release_year column.

In [18]:
update_maldita = """
    UPDATE tracks
    SET release_date = '2014'
    WHERE id = '74CSJTE5QQp1e4bHzm3wti';

    UPDATE tracks
    SET release_year = 2014
    WHERE id = '74CSJTE5QQp1e4bHzm3wti';
"""
c.executescript(update_maldita)
conn.commit()

#Check results
check_maldita = """
    SELECT *
    FROM tracks
    WHERE id = '74CSJTE5QQp1e4bHzm3wti';
"""
maldita = pd.read_sql_query(check_maldita, conn)
maldita

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,74CSJTE5QQp1e4bHzm3wti,Maldita sea la primera vez,19,233920,0,['Los Pincheira del Sur'],2014,0.66,0.79,2,-4.89,1,0.03,0.14,0.00,0.16,0.96,142.00,4,2014


#### E. Tempo

Next, let's check the tempo column. While the Spotify documentation doesn't give an explicit range for tempo [5], tempo shouldn't typically be 0, and it cannot be less than 0. Let's query for any distinct tempos less than or equal to 0.

In [19]:
check_tempo = """
    SELECT DISTINCT tempo
    FROM tracks
    WHERE tempo <= 0;
"""
execute_print(check_tempo, c)

(0.0,)


This pulled up 0, so let's look more closely at those records with a tempo of 0.

In [20]:
check_tempo_0 = """
    SELECT *
    FROM tracks
    WHERE tempo = 0;
"""
tempo_0 = pd.read_sql_query(check_tempo_0, conn)
tempo_0

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,2WTtJDfCUjAyCZHRSfCLDn,La Mina del Ford - Remasterizado,1,133947,0,['Ignacio Corsini'],1924-03-29,0.00,0.03,5,-24.89,1,0.00,1.00,0.77,0.08,0.00,0.00,0,1924
1,5tQsQaMquCXBREb1FNymi7,Oh Mujer Mujer - Remasterizado,0,170253,0,['Francisco Canaro'],1924-07-12,0.00,0.32,8,-15.97,0,0.00,1.00,0.96,0.38,0.00,0.00,0,1924
2,6VweawuAyN9Ad9ikn9f5W3,Shangai Bay - Remasterizado,0,129027,0,['Ignacio Corsini'],1924-03-29,0.00,0.08,6,-24.27,1,0.00,1.00,0.89,0.12,0.00,0.00,0,1924
3,1JUFU8XJRWgG9Ir8KlLvpW,Pause Track,0,4000,0,['Louis Armstrong'],1925,0.00,0.00,0,-60.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,1925
4,2fKnluaRpiiCFHWwKFkIgN,Pause Track,0,4000,0,['Louis Armstrong'],1925,0.00,0.00,0,-60.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,1925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,4THWkFVhSnDQzhh0khcVbp,Hidden Waterfall,33,422975,0,['White Noise Therapy'],2009-08-06,0.00,0.00,11,-13.81,0,0.00,0.20,1.00,0.32,0.00,0.00,0,2009
324,2LsevvtdFSKb8ztwOvQXf0,Brown Noise,50,570654,0,['Brown Noise'],2013-03-21,0.00,0.00,0,-16.52,1,0.00,0.12,0.43,0.47,0.00,0.00,0,2013
325,33wp91bmIk1XYn7aKxMone,Clean White Noise,37,72652,0,"['Spa', 'Spa, Relaxation and Dreams', 'Relax M...",2015-12-10,0.00,0.00,9,-8.33,1,0.00,0.11,0.33,0.66,0.00,0.00,0,2015
326,2jhzrWv6ut9eieePWaYRC8,Spa Rain,45,97220,0,['Rain Sounds'],2015-12-14,0.00,0.01,8,-19.90,1,0.00,0.00,0.81,0.41,0.00,0.00,0,2015


For the sake of brevity, not all 328 tracks are displayed, but all 328 of these tracks have danceability and time signature of 0, which would make sense if they do not in fact have a tempo. It is possible that some or all of these are ambient or noise tracks without a beat, as some of the names might indicate (e.g., "Brown Noise", "Clean White Noise"). Some of these are also clearly titled as pause tracks. All the tracks in this list also have valence of 0, which would indicate that they are all very negative. However, this could also just represent a lack of valence, which would make more sense if the tracks were simply noise or silent.

It is also possible that at least some of these have a tempo but are just missing tempo information altogether, possibly because Spotify had difficulty detecting it. For example, the first three rows in the table above are old recordings that are likely low quality. However, we cannot know this for sure without listening to all of these tracks. Given the evidence we have, it is reasonable to assume that these could be tracks where a tempo is either nonexistent or difficult to detect, and thus leaving the value at 0 would make the most sense here. If necessary, we can exclude tempos of 0 from any queries we perform.

#### F. Time signature

We also know now that we have some rows with time_signature equal to 0. The Spotify documentation says that it assigns a number from 3 to 7 for a track's time signature [5], so let's first look directly at the rows that have time_signature less than or equal to 0.

In [21]:
check_time_0 = """
    SELECT *
    FROM tracks
    WHERE time_signature <= 0;
"""
time_0 = pd.read_sql_query(check_time_0, conn)
time_0

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,2WTtJDfCUjAyCZHRSfCLDn,La Mina del Ford - Remasterizado,1,133947,0,['Ignacio Corsini'],1924-03-29,0.00,0.03,5,-24.89,1,0.00,1.00,0.77,0.08,0.00,0.00,0,1924
1,5tQsQaMquCXBREb1FNymi7,Oh Mujer Mujer - Remasterizado,0,170253,0,['Francisco Canaro'],1924-07-12,0.00,0.32,8,-15.97,0,0.00,1.00,0.96,0.38,0.00,0.00,0,1924
2,6VweawuAyN9Ad9ikn9f5W3,Shangai Bay - Remasterizado,0,129027,0,['Ignacio Corsini'],1924-03-29,0.00,0.08,6,-24.27,1,0.00,1.00,0.89,0.12,0.00,0.00,0,1924
3,1JUFU8XJRWgG9Ir8KlLvpW,Pause Track,0,4000,0,['Louis Armstrong'],1925,0.00,0.00,0,-60.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,1925
4,2fKnluaRpiiCFHWwKFkIgN,Pause Track,0,4000,0,['Louis Armstrong'],1925,0.00,0.00,0,-60.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,1925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,4THWkFVhSnDQzhh0khcVbp,Hidden Waterfall,33,422975,0,['White Noise Therapy'],2009-08-06,0.00,0.00,11,-13.81,0,0.00,0.20,1.00,0.32,0.00,0.00,0,2009
333,2LsevvtdFSKb8ztwOvQXf0,Brown Noise,50,570654,0,['Brown Noise'],2013-03-21,0.00,0.00,0,-16.52,1,0.00,0.12,0.43,0.47,0.00,0.00,0,2013
334,33wp91bmIk1XYn7aKxMone,Clean White Noise,37,72652,0,"['Spa', 'Spa, Relaxation and Dreams', 'Relax M...",2015-12-10,0.00,0.00,9,-8.33,1,0.00,0.11,0.33,0.66,0.00,0.00,0,2015
335,2jhzrWv6ut9eieePWaYRC8,Spa Rain,45,97220,0,['Rain Sounds'],2015-12-14,0.00,0.01,8,-19.90,1,0.00,0.00,0.81,0.41,0.00,0.00,0,2015


Again, not all rows are displayed, but the results include every track that have a tempo of 0, plus 9 tracks that have a positive tempo. Upon listening to a few of these tracks that have a positive tempo, it is unclear why there wouldn't be time signature information for the tracks, because they clearly have time signatures. As with tracks with tempo of 0, it is possible that Spotfiy just had a difficult time classifying these tracks for some reason and is missing the information. Discussion on the [Spotify Web API Github](https://github.com/spotify/web-api/issues/379) indicates that machine learning algorithms determine the time signature, and a computer might not always be able to detect a tempo like a human would. However, if we assume that at least some of the tracks with tempo of 0 are just noise or ambient, then there would not actually be any time signature to detect. As with tracks where tempo is 0, it makes the most sense to keep these as 0. If necessary, we can exclude tempos of 0 from any queries we perform.

Let's also check generally for songs having other out-of-range time signatures. First, time signatures greater than 7.

In [22]:
check_time_7 = """
    SELECT *
    FROM tracks
    WHERE time_signature > 7;
"""
time_7 = pd.read_sql_query(check_time_7, conn)
time_7

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year


There aren't any time signatures greater than 7. Next, time signatures greater than 0 but less than 3.

In [23]:
check_time_rest = """
    SELECT *
    FROM tracks
    WHERE time_signature > 0 AND time_signature < 3;
"""
time_rest = pd.read_sql_query(check_time_rest, conn)
time_rest

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],1922-06-01,0.69,0.26,0,-22.14,1,0.96,0.80,0.00,0.15,0.66,102.01,1,1922
1,0cC9CYjLRIzwchQ42xVnq6,Capítulo 1.23 - Banquero Anarquista,0,96600,0,['Fernando Pessoa'],1922-06-01,0.69,0.20,4,-24.26,0,0.96,0.75,0.00,0.20,0.48,78.45,1,1922
2,0grXU6GKVNCVMJbseA0Uhe,Capítulo 1.10 - Banquero Anarquista,0,95800,0,['Fernando Pessoa'],1922-06-01,0.70,0.21,2,-23.87,1,0.96,0.69,0.00,0.44,0.61,85.74,1,1922
3,0zyKYuXwmo8eR3BBxoxVEO,Capítulo 2.3 - Banquero Anarquista,0,113200,0,['Fernando Pessoa'],1922-06-01,0.66,0.21,0,-23.34,1,0.96,0.67,0.00,0.28,0.76,76.09,1,1922
4,15BEv0mKGqHHxyDAcLJ7ei,En la Huella del Querer - Remasterizado,0,150160,0,['Ignacio Corsini'],1922-03-29,0.37,0.18,4,-22.05,0,0.07,0.99,0.87,0.16,0.38,106.59,1,1922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6599,4fWWQCQE30yVtflYr5sUeo,"Scene 1: Psalty & Kids Meet, Greet & Check-In",20,142867,0,"['Psalty', 'Ernie Rettino', 'Debby Kerner Rett...",1985-06-01,0.65,0.15,10,-21.65,0,0.95,0.67,0.00,0.90,0.59,87.89,1,1985
6600,3a2ZFpNM9hrvI3Ce9HUB8N,Praise God From Whom All Blessings Flow,23,67133,0,"['Traditional', 'London Philharmonic Choir', '...",1988-01-01,0.21,0.06,8,-29.01,1,0.05,0.99,0.23,0.04,0.10,100.06,1,1988
6601,46YGAzyhQsY4IeN59QtqcF,Impossible,25,306027,0,['Laura Fygi'],1991-01-01,0.21,0.25,6,-14.37,1,0.03,0.82,0.00,0.14,0.08,94.59,1,1991
6602,58hyDWe9z3FzYfn7J28Yc7,Litany of the Saints,25,226773,0,['The Cathedral Singers'],1995-01-01,0.18,0.15,2,-20.93,1,0.03,0.97,0.14,0.19,0.04,96.23,1,1995


This brings up over 6000 rows. The displayed rows have a time signature of 1, but are there also rows with a time signature of 2? Let's check.

In [24]:
check_time_2 = """
    SELECT *
    FROM tracks
    WHERE time_signature = 2;
"""
time_2 = pd.read_sql_query(check_time_2, conn)
time_2

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year


This brings up no results, so there are just a large number of tracks that have a time signature of 1. Again, it is possible that Spotify had difficulty classifying these tracks. Since we don't have many clues about why these time signatures might be missing, one way we might be able to get more information is to create a dummy variable that is 1 if the track has a time signature of 1, then perform a logistic regression of this dummy variable on all the other variables in the data. Any large, statistically significant coefficients in the results might indicate that there is a pattern to which tracks have a time signature of 1 [7]. We cannot realistically include dummies for all artists here in the regression, and it is highly likely that there is some multicollinearity among the audio feature variables, so this will not be perfect. But it could still give us some clues.

In [25]:
#Read all tracks into a pandas DataFrame
select_all_tracks = """
    SELECT *
    FROM tracks;
"""
tracks = pd.read_sql_query(select_all_tracks, conn)

#Create a dummy variable that is 1 when time signature is 1
tracks['time_1'] = 0
tracks.loc[tracks['time_signature'] == 1, 'time_1'] = 1

#Perform logistic regression of dummy variable on other numeric variables
time_1_logit = smf.logit("""time_1 ~ danceability + energy + key + loudness + mode + speechiness
                            + acousticness + instrumentalness + liveness + valence + tempo + duration_ms
                            + explicit + release_year + popularity""", data=tracks)
time_1_logit.fit().summary()

Optimization terminated successfully.
         Current function value: 0.053377
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 time_1   No. Observations:               586672
Model:                          Logit   Df Residuals:                   586656
Method:                           MLE   Df Model:                           15
Date:                Wed, 01 Jun 2022   Pseudo R-squ.:                  0.1349
Time:                        13:33:57   Log-Likelihood:                -31315.
converged:                       True   LL-Null:                       -36197.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -2.7984      1.470     -1.904      0.057      -5.680       0.083
danceability        -3.2870      0.109    -30.171      0.000      -3.501      -3.073
energy              -1.4782      0.104    -14.158      0.000      -1.683      -1.274
key                 -0.0009      0.004     -0.238      0.811      -0.008       0.006
loudness             0.0022      0.003      0.655      0.512      -0.004       0.009
mode                -0.0457      0.027     -1.692      0.091      -0.099       0.007
speechiness          2.7462      0.052     53.319      0.000       2.645       2.847
acousticness         0.8883      0.059     15.014      0.000       0.772       1.004
instrumentalness    -0.0827      0.047     -1.749      0.080      -0.175       0.010
liveness             0.1139      0.070      1.634      0.102      -0.023       0.250
valence             -0.0458      0.071     -0.647      0.517      -0.185       0.093
tempo               -0.0027      0.000     -6.743      0.000      -0.003      -0.002
duration_ms      -3.404e-06   1.52e-07    -22.369      0.000    -3.7e-06   -3.11e-06
explicit             0.1195      0.090      1.322      0.186      -0.058       0.297
release_year         0.0004      0.001      0.545      0.586      -0.001       0.002
popularity          -0.0021      0.001     -1.990      0.047      -0.004   -3.14e-05
====================================================================================
"""

There are large, statistically significant coefficents for danceability, energy, speechiness, and acousticness. Songs with lower energy and danceability and higher speechiness and acousticness scores are more likely to have a time signature of 1. This seems consistent with the idea that Spotify has a difficult time identifying the time signature for songs with certain audio features, namely lower energy and danceability, higher speechiness and acousticness tracks.

This would make these missing time signatures missing at random, as the missing time signatures would depend on other audio features in the tracks. However, at least some of this data could also be missing not at random. Spotify picks a time signature from 3 to 7, but there are many other possible time signatures, including 11, 13, or mixed time signatures. Spotify very likely has a difficult time classifying these into its relatively limited range of 3 to 7.

So, time signature values of 1 are likely just missing data. Just over 1% of tracks are missing time signature data, which isn't huge but also isn't insignificant. If the missing data were random, we could possibly impute the most frequent time signature, which is probably 4. However, we think the data could be missing not at random, so imputing these missing time signatures with the mode could lead to bias. The simplest course of action, like with time signatures of 0, would be to just treat time signatures of 1 as a separate category [3], representing tracks that Spotify had difficulty classifying.

### 2. Missing data

As with the artists table, we might still have some blank or NULL values in our tracks table, so we should query the table for these records. Let's look for missing values in any column.

In [26]:
check_tracks_missing = """
    SELECT *
    FROM tracks
    WHERE name IS NULL OR name = ''
    OR popularity IS NULL OR popularity = ''
    OR duration_ms IS NULL OR duration_ms = ''
    OR explicit IS NULL OR explicit = ''
    OR artists IS NULL OR artists = ''
    OR release_date IS NULL OR release_date = ''
    OR danceability IS NULL OR danceability = ''
    OR energy IS NULL OR energy = ''
    OR key IS NULL OR key = ''
    OR loudness IS NULL OR loudness = ''
    OR mode IS NULL OR mode = ''
    OR speechiness IS NULL OR speechiness = ''
    OR acousticness IS NULL OR acousticness = ''
    OR instrumentalness IS NULL OR instrumentalness = ''
    OR liveness IS NULL OR liveness = ''
    OR valence IS NULL OR valence = ''
    OR tempo IS NULL OR tempo = ''
    OR time_signature IS NULL OR time_signature = ''
    OR release_year IS NULL OR release_year = '';
"""
tracks_missing = pd.read_sql_query(check_tracks_missing, conn)
tracks_missing

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,4iH7negBYMfj2z0wDNmgdx,,28,264973,0,[''],1994-01-01,0.51,0.58,0,-12.28,0,0.03,0.04,0.00,0.52,0.69,156.47,1,1994
1,04d5kbLvSAIBt3pGcljdhC,,0,184293,0,[''],1922-04-01,0.43,0.28,11,-11.97,1,0.05,0.99,0.26,0.29,0.58,135.66,4,1922
2,05tRkgyxVdwMePGqOXMDYU,,0,191587,0,[''],1922-04-01,0.34,0.19,0,-13.49,1,0.07,0.99,0.00,0.12,0.29,79.59,1,1922
3,0YAMRgAQH6tkTh4sWNXr8L,,0,191573,0,[''],1922-04-01,0.32,0.26,3,-13.61,0,0.05,0.99,0.77,0.52,0.53,68.68,3,1922
4,1K6MQQxmFpPb66ZnaiIpHX,,0,167602,0,[''],1922-04-01,0.56,0.28,1,-12.85,1,0.06,1.00,0.00,0.45,0.61,70.38,4,1922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,6OH9mz9aFbGlbf74cBwYWD,,2,209760,0,[''],1962-02-01,0.51,0.60,7,-4.67,0,0.06,0.77,0.00,0.69,0.80,91.17,4,1962
67,15RqFDA86slfzujSQMEX4i,,2,257280,0,[''],1962-02-01,0.61,0.61,5,-5.61,1,0.06,0.85,0.00,0.05,0.81,90.54,4,1962
68,0hKA9A2JPtFdg0fiMhyjQD,,6,194081,0,[''],1974-12-31,0.47,0.37,4,-12.93,0,0.15,0.97,0.00,0.14,0.77,94.06,4,1974
69,1kR4gIb7nGxHPI3D2ifs59,,26,289440,0,[''],1998-01-05,0.50,0.58,7,-9.46,0,0.06,0.69,0.00,0.07,0.73,138.39,4,1998


Again, not all rows are displayed, but the only columns that have entirely missing values are name and artists. Upon plugging some of these track IDs into URLs to search for the tracks directly on Spotify, they led to what appeared to be blank pages with no actual tracks to listen to. So, it is unclear if these are even accessible tracks in Spotify.

Let's join the tracks_artists table and see if the tracks that are missing names have a corresponding artist ID.

In [27]:
join_tracks_artists = """
    SELECT t.id, ta.artist_id
    FROM tracks AS t
    INNER JOIN tracks_artists AS ta
    ON t.id = ta.track_id
    WHERE t.name IS NULL OR t.name = '';
"""
tracks_artists_missing = pd.read_sql_query(join_tracks_artists, conn)
tracks_artists_missing

,id,artist_id
0,4iH7negBYMfj2z0wDNmgdx,0LyfQWJT6nXafLPZqxe9Of
1,04d5kbLvSAIBt3pGcljdhC,0LyfQWJT6nXafLPZqxe9Of
2,05tRkgyxVdwMePGqOXMDYU,0LyfQWJT6nXafLPZqxe9Of
3,0YAMRgAQH6tkTh4sWNXr8L,0LyfQWJT6nXafLPZqxe9Of
4,1K6MQQxmFpPb66ZnaiIpHX,0LyfQWJT6nXafLPZqxe9Of
...,...,...
66,6OH9mz9aFbGlbf74cBwYWD,0LyfQWJT6nXafLPZqxe9Of
67,15RqFDA86slfzujSQMEX4i,0LyfQWJT6nXafLPZqxe9Of
68,0hKA9A2JPtFdg0fiMhyjQD,0LyfQWJT6nXafLPZqxe9Of
69,1kR4gIb7nGxHPI3D2ifs59,0LyfQWJT6nXafLPZqxe9Of


All of these tracks match the same artist ID. Does this artist ID match an artist name on the artists table?

In [28]:
artists[artists['id'] == '0LyfQWJT6nXafLPZqxe9Of']

,id,followers,genres,name,popularity


This artist ID does not have a corresponding entry on the artists table, which is very odd. A quick Google search reveals that Spotify actually does have a profile for this ID called "Various Artists":

![Various Artists](./images/various_artists.png)

It isn't very clear, but it seems like this might be a catch-all that Spotify uses for songs by certain groups of artists. 

In terms of actually retrieving songs we can listen to, or joining data from the artists and genres tables, these rows are not very useful to us. However, the rest of the data for these rows looks valid, and the fact that Spotify actually has a page for this artist ID indicates that they could be valid tracks. So, rather than drop them outright and lose potentially valid data, we can just label these as missing track names.

In [29]:
label_tracks_missing = """
    UPDATE tracks
    SET name = 'Track Name Missing'
    WHERE name IS NULL OR name = '';
"""
c.execute(label_tracks_missing)
conn.commit()

#Check results
check_tracks = """
    SELECT *
    FROM tracks
    WHERE name = 'Track Name Missing'
"""
track_name_missing = pd.read_sql_query(check_tracks, conn)
track_name_missing

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
0,4iH7negBYMfj2z0wDNmgdx,Track Name Missing,28,264973,0,[''],1994-01-01,0.51,0.58,0,-12.28,0,0.03,0.04,0.00,0.52,0.69,156.47,1,1994
1,04d5kbLvSAIBt3pGcljdhC,Track Name Missing,0,184293,0,[''],1922-04-01,0.43,0.28,11,-11.97,1,0.05,0.99,0.26,0.29,0.58,135.66,4,1922
2,05tRkgyxVdwMePGqOXMDYU,Track Name Missing,0,191587,0,[''],1922-04-01,0.34,0.19,0,-13.49,1,0.07,0.99,0.00,0.12,0.29,79.59,1,1922
3,0YAMRgAQH6tkTh4sWNXr8L,Track Name Missing,0,191573,0,[''],1922-04-01,0.32,0.26,3,-13.61,0,0.05,0.99,0.77,0.52,0.53,68.68,3,1922
4,1K6MQQxmFpPb66ZnaiIpHX,Track Name Missing,0,167602,0,[''],1922-04-01,0.56,0.28,1,-12.85,1,0.06,1.00,0.00,0.45,0.61,70.38,4,1922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,6OH9mz9aFbGlbf74cBwYWD,Track Name Missing,2,209760,0,[''],1962-02-01,0.51,0.60,7,-4.67,0,0.06,0.77,0.00,0.69,0.80,91.17,4,1962
67,15RqFDA86slfzujSQMEX4i,Track Name Missing,2,257280,0,[''],1962-02-01,0.61,0.61,5,-5.61,1,0.06,0.85,0.00,0.05,0.81,90.54,4,1962
68,0hKA9A2JPtFdg0fiMhyjQD,Track Name Missing,6,194081,0,[''],1974-12-31,0.47,0.37,4,-12.93,0,0.15,0.97,0.00,0.14,0.77,94.06,4,1974
69,1kR4gIb7nGxHPI3D2ifs59,Track Name Missing,26,289440,0,[''],1998-01-05,0.50,0.58,7,-9.46,0,0.06,0.69,0.00,0.07,0.73,138.39,4,1998


### 3. Check for duplicates

Finally, as with the artists table, we should check for duplicate track IDs.

In [30]:
duplicate_tracks = """
    WITH duplicates AS (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY id ORDER BY id) row_num
        FROM tracks)
    SELECT *
    FROM duplicates
    WHERE row_num > 1;
"""
execute_print(duplicate_tracks, c)

No results


The query pulled up no duplicate track IDs.

## III. Conclusion

Now that we are done cleaning, we can close the connection. The database is now ready to perform queries on.

In [31]:
conn.close()

## IV. References

1. 'Get Artist'. Spotify for Developers: Web API Reference. https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artist.
2. Roy B. September 3, 2019, 'All About Missing Data Handling'. Towards Data Science. https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4.
3. Swalin A. January 30, 2018, 'How to Handle Missing Data'. Towards Data Science. https://towardsdatascience.com/all-about-missing-data-handling-b94b8b5d2184.
4. 'Get Track'. Spotify for Developers: Web API Reference. https://developer.spotify.com/documentation/web-api/reference/#/operations/get-track.
5. 'Get Track's Audio Features'. Spotify for Developers: Web API Reference. https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features.
6. Ay YE. April 2021, 'Spotify Dataset 1921-2020, 600k+ Tracks'. Kaggle. https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks?select=tracks.csv.
7. Buisson F (2021). Behavioral Data Analysis with R and Python. O’Reilly Media, Inc.